# MMB Model Frontiers based on a PI policy rule

## Optimization file only

If you are on the **monetaryPolicy** project, please use other file `PyLab.ipynb` for your analysis.

This cell sets up the notebook to import numpy, seaborn, pandas, matplotlib etc.

In [2]:
# Run this cell to set up the notebook.

# These lines import the Numpy, Pandas, Seaborn, Matplotlib modules.
import numpy as np
import pandas as pd
import seaborn as sns
import random
import matplotlib
import matplotlib.pyplot as plt

# Importing plotting libraries and styles
%matplotlib inline
plt.style.use('fivethirtyeight')

# For Pandas to ignore FutureWarning displays
import warnings
warnings.simplefilter('ignore', FutureWarning)

Install `matlab.engine` using this link:

https://www.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html

### The cell given below sets up MATLAB for the notebook
Source: https://sehyoun.com/blog/20180904_using-matlab-with-jupyter-notebook.html

In [3]:
import matlab.engine
import io
import scipy.io
from IPython.core.magic import register_cell_magic
ip = get_ipython()

out = io.StringIO()
err = io.StringIO()

# Setup matlab cell magic #
@register_cell_magic
def matlab_magic(line,cell):
    out.truncate(0)
    out.seek(0)
    err.truncate(0)
    err.truncate(0)
    raw = '''{line}.eval("""{cell}""", nargout=0, stdout=out, stderr=err)'''
    ip.run_cell(raw.format(line=line, cell=cell))
    print(out.getvalue())
    print(err.getvalue())
    
# Starting a MATLAB engine called eng
eng = matlab.engine.start_matlab()

**Note:** Change this to the file path on your computer.

In [4]:
# Adds the MMB.m as well as MMBOPT1.m and MMBOPT2.m folders to the MATLAB engine path"
eng.addpath(r'/Users/Desktop/monetaryPolicy/mmb-gui-mlab-2.3.2', nargout=0)
eng.addpath(r'/Users/Desktop/monetaryPolicy/mmb-gui-mlab-2.3.2/MMB_OPTIONS', nargout=0)
eng.addpath(r'/Users/Desktop/monetaryPolicy/scripts', nargout=0)

# Important:
The code below sets the coefficients and other data for the PID rule to work.

Check out the coefficients table here:

https://rishab231.github.io/img/coefficients.png

In [1]:
# This sets the coefficients of the monetary policy rule, there are 33 coefficients and len(coefficients) = 33
coefficients = [0, 0, 0, 0, 1.5/4, 1.5/4, 1.5/4, 1.5/4, 
                0, 0, 0, 0, 0, 0.5, 0, 0, 
                0, 0, 0, 0, 0, 0, 0, 0, 
                0, 0, 0, 0, 0, 0, 0, 1, 0.25]

# Number of the model you want to chooose, please exclude 69-79, 19-22, 27, 59, 65, 68, 81, 97, 98
modelNum = 1

scipy.io.savemat('variables.mat', dict(coefficients=coefficients, modelNumber = modelNum))

NameError: name 'scipy' is not defined

## **Important:** 
The cell below runs the MMB.m file

In [5]:
eng.MMB(nargout = 0)

### Functions defined to import data for:
* 4 IRF: Impulse Response Function Variables (outputgap, inflation, interest, output) and `modelName`
* All IRF Variables
* 4 ACF: Autocorrelation Function Variables (outputgap, inflation, interest, output)
* **Unconditional Variances**


In [6]:
def getModelName():
    irf_4 = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "IRF Mon. Pol. Shock      ")
    irf_4 = irf_4.T
    irf_headers = irf_4.iloc[0] # grab the first row for the header
    irf_4 = irf_4[1:] # take the data less the header row
    irf_4_stripped_headers = [myHeader.strip() for myHeader in np.array(irf_headers)] # removing trailing whitespaces
    irf_4.columns = irf_4_stripped_headers
    modelName = irf_4.columns.values[1]
    return modelName

def singleModel_irf4():
    irf_4 = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "IRF Mon. Pol. Shock      ")
    irf_4 = irf_4.T
    irf_headers = irf_4.iloc[0] # grab the first row for the header
    irf_4 = irf_4[1:] # take the data less the header row
    irf_4_stripped_headers = [myHeader.strip() for myHeader in np.array(irf_headers)] # removing trailing whitespaces
    irf_4.columns = irf_4_stripped_headers
    modelName = irf_4.columns.values[1]
    irf_4 = irf_4.iloc[:, [i for i in range(1, len(irf_4.columns.values), 2)]]
    irf_4.columns = ["OutputGap", "Inflation", "Interest", "Output"]
    irf_4 = irf_4.reset_index()
    irf_4.index.name = "Period"
    irf_4.drop('index', axis=1, inplace=True)
    return irf_4

def singleModel_allirf():
    old_irf_df = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "all IRFs Mon. Pol. Shock")
    all_irf = old_irf_df.T
    new_header = all_irf.iloc[0] # grab the first row for the header
    all_irf = all_irf[1:] # take the data less the header row
    stripped_headers = [myHeader.strip() for myHeader in np.array(new_header)] # removing trailing whitespaces
    all_irf.columns = stripped_headers # set the header row as the df header
    all_irf["c_t"] = all_irf.index
    all_irf.index = np.arange(0,21,1)
    all_irf.index.name = "Period"

    # This section rearranges the columns
    n = len(list(all_irf.columns.values))
    rearranged = [list(all_irf.columns.values)[-1]] + list(all_irf.columns.values)[:n-1]
    all_irf = all_irf[rearranged]
    return all_irf

def singleModel_acf():
    acf = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "ACF")
    acf = acf.T
    acf_headers = acf.iloc[0] # grab the first row for the header
    acf = acf[1:] # take the data less the header row
    acf_stripped_headers = [myHeader.strip() for myHeader in np.array(acf_headers)] # removing trailing whitespaces
    acf.columns = acf_stripped_headers
    acf = acf.iloc[:, [i for i in range(0, len(acf.columns.values), 2)]]
    acf.columns = ["OutputGap", "Inflation", "Interest", "Output"]
    acf = acf.reset_index()
    acf.index.name = "Period"
    acf.drop('index', axis=1, inplace=True)
    return acf

def unconditionalVariances():
    var4 = pd.read_csv("../mmb-gui-mlab-2.3.2/OUTPUT/variances.csv", names=["interest", "inflation", "outputgap", "output"])
    return var4

In [7]:
singleModel_irf4().head(3)

OutputGap  Inflation    Interest     Output
Period                                             
0               0          0           0          0
1       -0.823109 -0.0181086    0.561283  -0.823109
2       0.0414876 -0.0161234 -0.00344134  0.0414876

In [8]:
singleModel_acf().head(3)

OutputGap Inflation  Interest    Output
Period                                        
0              1         1         1         1
1       0.661577  0.716781  0.755958  0.730718
2       0.375576  0.478675  0.590853  0.520949


Calculating `unconditionalVariances` for different models, in an array of `modelNums`

In [9]:
def myVariance(modelNums, coeff):
    variances = dict()
    for modelNum in modelNums:
        eng.MMB(nargout = 0)
        scipy.io.savemat('variables.mat', dict(coefficients=coeff, modelNumber = modelNum))
        modelName = getModelName()
        variances[modelName] = unconditionalVariances().values.tolist()[0]
    return variances

Calculating unconditional variances for different rules

In [10]:
myVariance([1], coefficients)

{'NK_RW97': [0.079687, 0.06191, 0.38900999999999997, 1.1702]}

## Hawkins PI Optimization (Williams Paper)

`selected_coeff` has $\beta^{(\pi_t)}, \beta^{(\pi_{t-1})}, \beta^{(y_t)}, \beta^{(y_{t-1})}$ 

So, the policy rule becomes:

$r_t = r_{t-1} + \beta^{(\pi_t)}\pi_t + \beta^{(\pi_{t-1})}\pi_{t-1} + \beta^{(y_t)}y_t + \beta^{(y_{t-1})}y_{t-1}$

In [11]:
PID_final_inflation_variance = 0
PID_final_output_gap_variance = 0
PID_final_interest_variance = 0
currentPIDLoss = 0

def myPID(selected_coeff, lambdaVal, coeffInterest, modelNum, rVarTarget):
    global PID_final_inflation_variance
    global PID_final_output_gap_variance
    global PID_final_interest_variance
    global currentPIDLoss

    coefficients = [1, 0, 0, 0, 
                    abs(selected_coeff[0])/4, 
                    (abs(selected_coeff[0])+selected_coeff[1])/4, 
                    (abs(selected_coeff[0])+selected_coeff[1])/4, 
                    (abs(selected_coeff[0])+selected_coeff[1])/4, 
                    selected_coeff[1]/4, 
                    0, 0, 0, 0, 
                    selected_coeff[2], selected_coeff[3], 0, 
                    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0.25]    
    scipy.io.savemat('variables.mat', dict(coefficients=coefficients, modelNumber = modelNum)) # Input for MMB.
    eng.MMB(nargout = 0)
    
    interest_rate_variance = unconditionalVariances()['interest'][0]
    inflation_variance = unconditionalVariances()['inflation'][0]
    output_gap_variance = unconditionalVariances()['outputgap'][0]
    
    PID_final_inflation_variance = inflation_variance
    PID_final_output_gap_variance = output_gap_variance
    PID_final_interest_variance = interest_rate_variance
    
    PID_loss = ((interest_rate_variance - rVarTarget)**2 * coeffInterest 
                + inflation_variance * lambdaVal
                + output_gap_variance * (1 - lambdaVal))
    
    currentPIDLoss = PID_loss
    print("lambda =", lambdaVal, "current_loss =", currentPIDLoss)
    
    return PID_loss

In [12]:
def createFrontier(modelNum):
    rVarTarget = 16.0 # Target rate variance from Williams paper.
    taylor_coeff = [1.5, 0, 0.5, 0] # Taylor rule coefficients.
    hamilton_coeff = [1.42, -1.20, 0.5, -0.48] # Hamilton fit coefficients.
    weights = np.arange(0, 1.01, .25) # Values for lambda
    interest_coeff = 1.0 # Coefficient on the interest_rate loss term
    
    mylambdatemp = 0.0 # Something to put in the lambda variable when calculating the target rate variance.
    # Run with Hamilton coefficients to generate target rate variance.
    myfoo = myPID(hamilton_coeff, mylambdatemp, interest_coeff, modelNum, rVarTarget)
    rVarTarget = unconditionalVariances()['interest'][0] # Reassign target rate variance.
    print("The target rate variance is", rVarTarget) # Print assigned rate variance.
    
    PID_inflation_variances = []
    PID_output_gap_variances = []
    PID_interest_variances = []
    PID_loss_of_lambda = []
    PID_coefficients_array = []

    # Starting with the Taylor rule coefficients
    #current_coeff = taylor_coeff
    
    # Starting with the Hamilton rule coefficients
    current_coeff = hamilton_coeff

    for lambda_value in weights:
        PID_result = scipy.optimize.minimize(myPID, current_coeff, \
                                    args=(lambda_value, interest_coeff, modelNum, rVarTarget), method='Nelder-Mead')
        current_coeff = PID_result.x
        PID_coefficients_array.append(current_coeff)
        PID_inflation_variances.append(PID_final_inflation_variance)
        PID_output_gap_variances.append(PID_final_output_gap_variance)
        PID_interest_variances.append(PID_final_interest_variance)
        PID_loss_of_lambda.append(currentPIDLoss)
    
    # The code below plots the frontier
    nameOfModel = getModelName()
    
    SD_inflation_scatter = np.sqrt(np.asarray(PID_inflation_variances))
    SD_output_gap_scatter = np.sqrt(np.asarray(PID_output_gap_variances))

    fig, ax = plt.subplots()
    ax.scatter(SD_inflation_scatter, SD_output_gap_scatter, color="red")

    for i in range(0, len(weights)):
        ax.annotate(weights[i], (SD_inflation_scatter[i], SD_output_gap_scatter[i]))

    ax.set_xlabel('$\sigma_{\pi}$', fontsize=10)
    ax.set_ylabel('$\sigma_{y}$', fontsize=10)
    ax.set_title('Policy Frontiers for Different Weights, Model: ' + nameOfModel, fontsize=14)
    saveFileName = nameOfModel + '.pdf'
    plt.savefig(saveFileName, bbox_inches='tight')
    
    # The code below saves the results of the all the optimizations with the appropriate lambdas into a DataFrame
    results = dict()
    results['lambdas'] = weights
    results['inflation_variance'] = PID_inflation_variances
    results['output_gap_variance'] = PID_output_gap_variances
    results['interest_variance'] = PID_interest_variances
    results['loss'] = PID_loss_of_lambda
    results['coefficients'] = PID_coefficients_array
    results_df = pd.DataFrame.from_dict(results)
    
    # Saving DataFrame to nameOfModel.csv file
    results_df.to_csv(nameOfModel + ".csv", index=False)
    
    return results_df

In [13]:
m15results = createFrontier(15)

lambda = 0.0 current_loss = 95.195944
The target rate variance is 22.712
lambda = 0.0 current_loss = 50.145
lambda = 0.0 current_loss = 49.301324
lambda = 0.0 current_loss = 83.28424400000003
lambda = 0.0 current_loss = 41.978401
lambda = 0.0 current_loss = 98.97032400000003
lambda = 0.0 current_loss = 43.530401
lambda = 0.0 current_loss = 39.797743999999994
lambda = 0.0 current_loss = 32.041241
lambda = 0.0 current_loss = 32.382321000000005
lambda = 0.0 current_loss = 32.86604899999999
lambda = 0.0 current_loss = 28.312943999999998
lambda = 0.0 current_loss = 43.439944
lambda = 0.0 current_loss = 44.444049000000014
lambda = 0.0 current_loss = 39.997009
lambda = 0.0 current_loss = 28.859889000000006
lambda = 0.0 current_loss = 43.94340900000001
lambda = 0.0 current_loss = 28.986881000000004
lambda = 0.0 current_loss = 30.41559999999999
lambda = 0.0 current_loss = 26.4956
lambda = 0.0 current_loss = 30.891728999999994
lambda = 0.0 current_loss = 26.072644
lambda = 0.0 current_loss = 29.

MatlabExecutionError: 
  File /Users/rishabsrivastava/Desktop/monetaryPolicy/mmb-gui-mlab-2.3.2/MMB.m, line 15, in MMB
'../scripts/variables.mat' is not found in the current folder or on the MATLAB path, but exists in:
    /Users/rishabsrivastava/Desktop/monetaryPolicy/mmb-gui-mlab-2.3.2

Change the MATLAB current folder or add its folder to the MATLAB path.


In [14]:
m15results

lambdas  inflation_variance  output_gap_variance  interest_variance  \
0    0.000             32.2720               13.906             16.170   
1    0.125             22.6760               14.458             16.188   
2    0.250             18.0110               15.485             16.224   
3    0.375             14.9090               16.866             16.248   
4    0.500             12.5520               18.685             16.257   
5    0.625             10.4090               21.441             16.259   
6    0.750              8.2562               26.321             16.241   
7    0.875              5.9515               37.099             16.193   
8    1.000              3.4858               88.649             16.069   

      losses                                       coefficients  
0  13.934900  [-0.22051918628999112, 0.0013264097158561397, ...  
1  15.520594  [-0.2700526295155092, 0.0013308918150854623, 0...  
2  16.166676  [-0.30571205625143016, 0.0011227951134167862, ...  
3  16.193629  [-0.3368461663245146, 0.0011298881515955676, 0...  
4  15.684549  [-0.36634825765828977, 0.0011447401706699016, ...  
5  14.613081  [-0.3994760140949899, 0.0011044817808612858, 0...  
6  12.830481  [-0.4418438174574822, 0.0010697404039113796, 0...  
7   9.882186  [-0.5054289602149964, 0.0010652964560608578, 0...  
8   3.490561  [-0.6681029822357823, 0.0014506820306309294, 3...

## Exploration of EuroArea Model Frontiers

In [12]:
Est_EA_Models = [34, 35, 36, 37, 38, 39, 40, 41, 57, 68, 82, 94]

## Constrained Optimization

In [13]:
PID_final_inflation_variance = 0
PID_final_output_gap_variance = 0
PID_final_interest_variance = 0
currentPIDLoss = 0

def constrainedPID(selected_coeff, lambdaVal, modelNum, rVarTarget):
    global PID_final_inflation_variance
    global PID_final_output_gap_variance
    global PID_final_interest_variance
    global currentPIDLoss
    global rateDifference

    coefficients = [1, 0, 0, 0, 
                    abs(selected_coeff[0])/4, 
                    (abs(selected_coeff[0])+selected_coeff[1])/4, 
                    (abs(selected_coeff[0])+selected_coeff[1])/4, 
                    (abs(selected_coeff[0])+selected_coeff[1])/4, 
                    selected_coeff[1]/4, 
                    0, 0, 0, 0, 
                    selected_coeff[2], selected_coeff[3], 0, 
                    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0.25]    
    scipy.io.savemat('variables.mat', dict(coefficients=coefficients, modelNumber = modelNum)) # Input for MMB.
    eng.MMB(nargout = 0)
    
    interest_rate_variance = unconditionalVariances()['interest'][0]
    inflation_variance = unconditionalVariances()['inflation'][0]
    output_gap_variance = unconditionalVariances()['outputgap'][0]
    
    PID_final_inflation_variance = inflation_variance
    PID_final_output_gap_variance = output_gap_variance
    PID_final_interest_variance = interest_rate_variance
    
    PID_loss = inflation_variance * lambdaVal + output_gap_variance * (1 - lambdaVal)
    currentPIDLoss = PID_loss
    
    # Important: Redefining the variable returned by the constraint function(s)
    rateDifference = rVarTarget - interest_rate_variance
    
    print("lambda =", lambdaVal, "current_loss =", currentPIDLoss, "interest_variance =", interest_rate_variance)
    
    return PID_loss

In [14]:
from scipy.optimize import fmin_cobyla as mini

rVarTarget = 16.0 # Target rate variance from Williams paper.
taylor_coeff = [1.5, 0, 0.5, 0] 
hamilton_coeff = [1.42, -1.20, 0.5, -0.48]
weights = np.arange(0, 1.01, .25) # Values for lambda
interest_coeff = 1.0 # Coefficient on the interest_rate loss term
modelNum = 15

# Run with Hamilton coefficients to generate target rate variance.
mylambdatemp = 0.0 # Something to put in the lambda variable when calculating the target rate variance.
myfoo = myPID(hamilton_coeff, mylambdatemp, interest_coeff, modelNum, rVarTarget)
rVarTarget = unconditionalVariances()['interest'][0] # Assign target rate variance.
print("The target rate variance is", rVarTarget)
print("Allowed interest rate variance is", (rVarTarget - 0.05*rVarTarget, rVarTarget + 0.05*rVarTarget))

# Important, these are the constraint function which should always be non-negative
# Note: 2 constraint functions to create equality constraint
def lowerbound(current_coeff, lambda_value, modelNum, rVarTarget):
    return -1*rateDifference + 0.05 * rVarTarget

def upperbound(current_coeff, lambda_value, modelNum, rVarTarget):
    return 0.05 * rVarTarget + rateDifference

# Assuming starting interest_rate_variance to be 0
rateDifference = rVarTarget - 0

# To store information
PID_inflation_variances = []
PID_output_gap_variances = []
PID_interest_variances = []
PID_loss_of_lambda = []
PID_coefficients_array = []

# Starting with the Taylor rule coefficients
#current_coeff = taylor_coeff

# Starting with the Hamilton rule coefficients
current_coeff = hamilton_coeff

for lambda_value in weights:
    PID_result_cons = mini(constrainedPID, current_coeff, args=(lambda_value, modelNum, rVarTarget), cons=[lowerbound, upperbound]) 
    # Append to all the lists to store information for this lambda_value
    break

lambda = 0.0 current_loss = 95.195944
The target rate variance is 22.712
lambda = 0.0 current_loss = 50.145 interest_variance = 22.712
lambda = 0.0 current_loss = 40.455 interest_variance = 44.548
lambda = 0.0 current_loss = 41.123999999999995 interest_variance = 64.021
lambda = 0.0 current_loss = 10.886 interest_variance = 98.34
lambda = 0.0 current_loss = 10.075 interest_variance = 182.95
lambda = 0.0 current_loss = 14.693 interest_variance = 68.557
lambda = 0.0 current_loss = 1050.1 interest_variance = 10893
lambda = 0.0 current_loss = 10.343 interest_variance = 154.38
lambda = 0.0 current_loss = 11.377 interest_variance = 97.584
lambda = 0.0 current_loss = 9.9557 interest_variance = 100.4
lambda = 0.0 current_loss = 10.384 interest_variance = 74.294
lambda = 0.0 current_loss = 11.765999999999998 interest_variance = 52.214
lambda = 0.0 current_loss = 10.321 interest_variance = 76.891
lambda = 0.0 current_loss = 11.827 interest_variance = 51.958999999999996
lambda = 0.0 current_loss 

MatlabExecutionError: 
  File /Users/rishabsrivastava/Desktop/monetaryPolicy/mmb-gui-mlab-2.3.2/MMB.m, line 15, in MMB
'../scripts/variables.mat' is not found in the current folder or on the MATLAB path, but exists in:
    /Users/rishabsrivastava/Desktop/monetaryPolicy/mmb-gui-mlab-2.3.2

Change the MATLAB current folder or add its folder to the MATLAB path.


In [18]:
kp0901 = PID_final_interest_variance
#PID_final_output_gap_variance = 0
#PID_final_interest_variance = 0
#currentPIDLoss

In [22]:
kp0901 - 0.050 * kp0901, kp0901 + .050 * kp0901

(21.639100000000003, 23.916900000000002)

In [ ]:
intrate > rvar - lower
intrate - rvar + lower > 0
-1 * rdiff + lower > 0

intrate < rvar + upper
upper + rvar - intrate > 0
upper + rdiff > 0